In [35]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import models
import importlib
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt
K = tf.keras.backend
import string
import keras

Using TensorFlow backend.


In [2]:
importlib.reload(models)

<module 'models' from '/Users/varun/School/CS182/finalproj/CS182-Spring2020-NLP-Project/models.py'>

In [3]:
df = pd.read_csv("dataset-binary.csv")
df = df.dropna()
clean_df = pd.read_csv("clean_large.csv")
clean_df = clean_df.dropna()
ultra_clean_df = pd.read_csv("ultra_clean.csv")
ultra_clean_df = ultra_clean_df.dropna()

In [4]:
train_df = df.iloc[:int(len(df) * .8)]
test_df = df.iloc[int(len(df) * .8):]
train_df_c = clean_df.iloc[:int(len(clean_df) * .8)]
test_df_c = clean_df.iloc[int(len(clean_df) * .8):]
train_df_u = ultra_clean_df.iloc[:int(len(ultra_clean_df) * .8)]
test_df_u = ultra_clean_df.iloc[int(len(ultra_clean_df) * .8):]

In [5]:
def custom_accuracy(y_true, y_pred):
        return K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32)) / 32

def distance(y_true, y_pred):
        total_corr = K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))
        return K.sum(K.cast(K.abs(K.argmax(y_true) - K.argmax(y_pred)), dtype = tf.int32)) / (32 - total_corr)
    
def sum_metrics(y_true, y_pred):
        acc = (32 - K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))) / 32
        total_corr = K.sum(K.cast(K.equal(K.argmax(y_true), K.argmax(y_pred)), dtype = tf.int32))
        return acc + K.sum(K.cast(K.abs(K.argmax(y_true) - K.argmax(y_pred)), dtype = tf.int32)) / (32 - total_corr)

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 5000, oov_token = '<OOV>')
tokenizer.fit_on_texts(train_df["text"])

In [122]:
bwcm_65_probs_dr_ft = tf.keras.models.load_model("bwcm_65_probs_dr_ft.h5", 
                                            custom_objects = {'custom_accuracy' : custom_accuracy, 
                                                             'distance' : distance,
                                                             'sum_metrics' : sum_metrics})

In [124]:
layers = [i for i in bwcm_65_probs_dr_ft.layers]

In [94]:
bwcm_65_probs_dr_ft.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 200)         1000000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, None, 200)         0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, None, 100)         100400    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 100)               45600     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 505       
Total params: 1,146,505
Trainable params: 1,146,505
Non-trainable params: 0
_________________________________________________________________


In [125]:
new_layers = layers[:3] + [tf.keras.layers.Bidirectional(tf.keras.layers.GRU(50))] + layers[4:]

In [126]:
for i in range(len(new_layers)):
    if i < 3:
        new_layers[i].trainable = False

In [127]:
model = tf.keras.Sequential()
for i in new_layers:
    model.add(i)

In [128]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy', custom_accuracy, distance, sum_metrics])

In [88]:
training_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train_df_u["text"][:8000]), 
                                                                maxlen = 250, padding = 'post', truncating = 'post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_df["text"][45000:47000]), 
                                                                maxlen = 250, padding = 'post', truncating = 'post')

In [89]:
training_stars = train_df_u[["stars_1.0", "stars_2.0", "stars_3.0", "stars_4.0", "stars_5.0"]].values[:8000]
test_stars = test_df[["stars_1.0", "stars_2.0", "stars_3.0", "stars_4.0", "stars_5.0"]].values[45000:47000]

In [99]:
history = model.fit(training_padded, models.stars_to_probs(training_stars, .65), batch_size = 32, 
                    epochs = 5, validation_data = (test_padded, test_stars), verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 - 294s - loss: 1.2806 - accuracy: 0.5878 - custom_accuracy: 0.5878 - distance: 1.6229 - sum_metrics: 2.0352 - val_loss: 0.9838 - val_accuracy: 0.6780 - val_custom_accuracy: 0.6726 - val_distance: 1.4347 - val_sum_metrics: 1.7621
Epoch 2/5
8000/8000 - 286s - loss: 1.2186 - accuracy: 0.6210 - custom_accuracy: 0.6210 - distance: 1.4921 - sum_metrics: 1.8711 - val_loss: 1.0060 - val_accuracy: 0.6555 - val_custom_accuracy: 0.6503 - val_distance: 1.4158 - val_sum_metrics: 1.7655
Epoch 3/5
8000/8000 - 291s - loss: 1.2047 - accuracy: 0.6367 - custom_accuracy: 0.6367 - distance: 1.4823 - sum_metrics: 1.8455 - val_loss: 0.9582 - val_accuracy: 0.6675 - val_custom_accuracy: 0.6622 - val_distance: 1.4089 - val_sum_metrics: 1.7467
Epoch 4/5
8000/8000 - 290s - loss: 1.1989 - accuracy: 0.6386 - custom_accuracy: 0.6386 - distance: 1.4748 - sum_metrics: 1.8362 - val_loss: 0.9888 - val_accuracy: 0.6660 - val_custom_accuracy: 0.6607 - val

In [100]:
model.save("bwcm_65_probs_dr_ft_2.h5")

In [101]:
training_padded_2 = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train_df["text"][50000:65000]), 
                                                                maxlen = 250, padding = 'post', truncating = 'post')
preds = bwcm_65_probs_dr_ft.predict(training_padded_2, verbose = 1)

15000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [102]:
trues = train_df[["stars_1.0", "stars_2.0", "stars_3.0", "stars_4.0", "stars_5.0"]].values[50000:65000]

In [110]:
true_texts = train_df["text"][50000:65000]
wrong_texts = []
wrong_texts_stars = []
for i in range(15000):
    if np.argmax(preds[i]) != np.argmax(trues[i]):
        wrong_texts.append(true_texts.iloc[i])
        wrong_texts_stars.append(trues[i])

In [118]:
wrong_texts_stars = np.array(wrong_texts_stars)[:4000]
wrong_texts = wrong_texts[:4000]

In [129]:
training_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(wrong_texts), 
                                                                maxlen = 250, padding = 'post', truncating = 'post')

In [134]:
history = model.fit(training_padded[:3000], wrong_texts_stars[:3000], batch_size = 32, 
                    epochs = 5, validation_data = (training_padded[3000:], wrong_texts_stars[3000:]), verbose=2)

Train on 3000 samples, validate on 1000 samples
Epoch 1/5
3000/3000 - 137s - loss: 1.1848 - accuracy: 0.5253 - custom_accuracy: 0.5239 - distance: 1.5785 - sum_metrics: 2.0545 - val_loss: 1.1053 - val_accuracy: 0.5480 - val_custom_accuracy: 0.5352 - val_distance: 1.4585 - val_sum_metrics: 1.9234
Epoch 2/5
3000/3000 - 129s - loss: 1.1658 - accuracy: 0.5327 - custom_accuracy: 0.5312 - distance: 1.5719 - sum_metrics: 2.0407 - val_loss: 1.0933 - val_accuracy: 0.5490 - val_custom_accuracy: 0.5361 - val_distance: 1.4677 - val_sum_metrics: 1.9316
Epoch 3/5
3000/3000 - 133s - loss: 1.1604 - accuracy: 0.5433 - custom_accuracy: 0.5419 - distance: 1.5756 - sum_metrics: 2.0337 - val_loss: 1.0816 - val_accuracy: 0.5620 - val_custom_accuracy: 0.5488 - val_distance: 1.5173 - val_sum_metrics: 1.9685
Epoch 4/5
3000/3000 - 126s - loss: 1.1432 - accuracy: 0.5460 - custom_accuracy: 0.5445 - distance: 1.5978 - sum_metrics: 2.0532 - val_loss: 1.0658 - val_accuracy: 0.5590 - val_custom_accuracy: 0.5459 - val

In [135]:
model.save("ft.h5")

In [21]:
m1s = []
m2s = []
joint_fuck = []
for i in range(len(training_padded)):
    if i % 500 == 0:
        print(i)
    m1_pred = np.argmax(model1.predict(np.array(training_padded[i]))[0])
    m2_pred = np.argmax(model2.predict(np.array(training_padded[i]))[0])
    corr = np.argmax(training_stars[i])
    if m1_pred == corr and m2_pred != corr:
        m1s.append((training_padded[i], training_stars[i]))
    elif m1_pred != corr and m2_pred == corr:
        m2s.append((training_padded[i], training_stars[i]))
    elif m1_pred != corr and m2_pred != corr:
        joint_fuck.append((training_padded[i], training_stars[i]))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [26]:
m1s_r = m1s[:350]
m2s_r = m2s[:350]
jf_r = joint_fuck[:350]

In [27]:
r = m1s_r + m2s_r + jf_r

In [25]:
len(m2s)

391

In [14]:
model1 = tf.keras.models.load_model("bwcm_9_probs_dr_3.h5", 
                                            custom_objects = {'custom_accuracy' : custom_accuracy, 
                                                             'distance' : distance,
                                                             'sum_metrics' : sum_metrics})

In [15]:
model2 = tf.keras.models.load_model("bwcm_9_probs_dr_e_2.h5", 
                                            custom_objects = {'custom_accuracy' : custom_accuracy, 
                                                             'distance' : distance,
                                                             'sum_metrics' : sum_metrics})

In [28]:
layers1 = [i for i in model1.layers]

In [29]:
layers2 = [i for i in model2.layers]

In [30]:
for i in layers1:
    i.trainable = False
for i in layers2:
    i.trainable = False

In [31]:
inp = tf.keras.Input(shape = (250,))
x1 = layers1[0](inp)
x2 = layers2[0](inp)
x1 = layers1[1](x1)
x2 = layers2[1](x2)
x1 = layers1[2](x1)
x2 = layers2[2](x2)
#x1 = layers1[3](x1)
#x2 = layers1[3](x2)
conc = tf.keras.layers.concatenate([x1, x2])
out = tf.keras.layers.Dense(5, activation = 'softmax')(conc)
model = tf.keras.Model(inputs = inp, outputs = out)

In [16]:
training_reviews, training_stars, test_reviews, test_stars = models.get_fit_samples(train_df, test_df, 10000)

In [20]:
training_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(training_reviews), 
                                                                maxlen = 250, padding = 'post', truncating = 'pre')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_reviews), 
                                                                maxlen = 250, padding = 'post', truncating = 'pre')

In [32]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy', custom_accuracy, distance, sum_metrics])

In [33]:
model.fit(np.array([i[0] for i in r] + [i for i in training_padded[:350]]), 
          np.array([i[1] for i in r] + [i for i in training_stars[:350]]), batch_size = 32, 
                    epochs = 5, validation_data = (test_padded, test_stars), verbose=2)

Train on 1400 samples, validate on 2000 samples
Epoch 1/5
1400/1400 - 75s - loss: 1.6624 - accuracy: 0.2643 - custom_accuracy: 0.2628 - distance: 2.3073 - sum_metrics: 3.0445 - val_loss: 1.0868 - val_accuracy: 0.6590 - val_custom_accuracy: 0.6538 - val_distance: 1.5489 - val_sum_metrics: 1.8951
Epoch 2/5
1400/1400 - 67s - loss: 1.0331 - accuracy: 0.6643 - custom_accuracy: 0.6605 - distance: 1.6807 - sum_metrics: 2.0201 - val_loss: 0.7856 - val_accuracy: 0.7215 - val_custom_accuracy: 0.7158 - val_distance: 1.6270 - val_sum_metrics: 1.9113
Epoch 3/5
1400/1400 - 68s - loss: 0.8808 - accuracy: 0.6757 - custom_accuracy: 0.6719 - distance: 1.6885 - sum_metrics: 2.0167 - val_loss: 0.7261 - val_accuracy: 0.7300 - val_custom_accuracy: 0.7242 - val_distance: 1.5520 - val_sum_metrics: 1.8277
Epoch 4/5
1400/1400 - 71s - loss: 0.8232 - accuracy: 0.6879 - custom_accuracy: 0.6839 - distance: 1.5897 - sum_metrics: 1.9058 - val_loss: 0.7042 - val_accuracy: 0.7325 - val_custom_accuracy: 0.7267 - val_dis

In [34]:
model.save("conc4.h5")

In [36]:
file = open("tokenizer.txt")

In [37]:
x = list(file)[0]

In [38]:
t = keras.preprocessing.text.tokenizer_from_json(x)